In [1]:
try:
    sc.stop()
except:
    print("no sc to stop")

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, HiveContext

conf = SparkConf().setAppName('SparkRDDSession').setMaster('local[4]')
sc = SparkContext.getOrCreate(conf = conf)


In [2]:
sc

<SparkContext master=local[4] appName=SparkRDDSession>

In [5]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [6]:
# hive integration with spark session
spark = (SparkSession.builder.appName('pysession'))

In [61]:
import numpy as np

B = sc.parallelize(np.random.randint(0,10, size=1000))
lst = B.collect()

In [8]:
for i in lst:
    print(i, end=', ')

6, 1, 6, 2, 8, 7, 2, 9, 9, 7, 8, 8, 7, 3, 1, 7, 7, 8, 5, 7, 6, 4, 2, 7, 3, 5, 1, 9, 8, 3, 3, 7, 8, 9, 1, 3, 0, 0, 7, 9, 2, 4, 4, 9, 2, 3, 5, 7, 7, 7, 9, 6, 3, 5, 9, 2, 1, 9, 7, 2, 2, 2, 7, 1, 1, 8, 7, 4, 7, 1, 0, 7, 6, 3, 1, 3, 9, 3, 7, 8, 9, 7, 9, 2, 4, 2, 1, 2, 3, 1, 1, 7, 9, 1, 5, 5, 8, 8, 8, 7, 7, 2, 3, 1, 9, 0, 4, 5, 3, 9, 7, 3, 6, 5, 3, 7, 6, 5, 7, 2, 3, 9, 3, 1, 8, 4, 8, 2, 9, 9, 0, 4, 8, 5, 2, 4, 9, 6, 8, 6, 6, 2, 2, 5, 3, 6, 2, 5, 9, 5, 2, 6, 7, 2, 4, 9, 6, 4, 9, 1, 6, 4, 2, 1, 5, 5, 2, 7, 6, 1, 9, 4, 4, 9, 8, 2, 4, 5, 8, 8, 4, 8, 0, 6, 5, 1, 6, 4, 1, 4, 5, 8, 5, 2, 3, 1, 1, 6, 7, 2, 8, 8, 8, 3, 1, 3, 0, 8, 1, 2, 2, 7, 5, 1, 3, 1, 0, 2, 2, 9, 2, 9, 8, 3, 6, 0, 6, 5, 0, 5, 8, 0, 5, 0, 8, 6, 9, 4, 5, 8, 6, 2, 5, 9, 8, 0, 5, 5, 9, 3, 3, 6, 1, 8, 0, 1, 4, 9, 2, 5, 1, 3, 5, 7, 5, 0, 1, 2, 4, 0, 3, 8, 9, 1, 7, 8, 0, 0, 3, 7, 1, 9, 1, 6, 5, 6, 8, 1, 4, 5, 8, 9, 2, 0, 1, 6, 1, 0, 0, 0, 3, 4, 7, 5, 2, 4, 1, 3, 6, 8, 8, 9, 4, 2, 5, 4, 9, 9, 8, 5, 7, 8, 6, 5, 8, 2, 4, 5, 7, 1, 7, 9, 8, 5

In [77]:
%%time
squares = B.map(lambda x:x*x)
summation = squares.reduce(lambda x,y:x+y)

CPU times: user 0 ns, sys: 12 ms, total: 12 ms
Wall time: 254 ms


In [63]:
summation

28432

In [64]:
###### Merge both syntaxes - sequential chaining

In [79]:
%%time
B.map(lambda x:x*x).reduce(lambda x,y:x+y)

CPU times: user 0 ns, sys: 12.1 ms, total: 12.1 ms
Wall time: 195 ms


28432

#### pipelined execution
* less memory required - intermediate results are not stored
* faster at scale - only one pass through input RDD

In [80]:
# Reduce

In [81]:
list_1 = np.random.randint(0,50,30)
rdd_1 = sc.parallelize(list_1)

In [82]:
rdd_1_distinct = rdd_1.distinct()

In [83]:
rdd_1_distinct.collect()

[48,
 20,
 24,
 4,
 44,
 12,
 36,
 1,
 37,
 33,
 18,
 34,
 6,
 22,
 10,
 2,
 30,
 19,
 35,
 39,
 47,
 7,
 15]

In [85]:
rdd_1_distinct.reduce(lambda x,y: x+y)

543

In [87]:
rdd_1_distinct.reduce(lambda x,y: x if x>y else y)

48

In [88]:
# ReduceByKey


In [89]:
data = [
        ('project', 1),
        ('alice', 1),
        ('adventures', 1),
        ('in', 1),
        ('wonderland', 1),
        ('alice', 1),
        ('adventures', 1),
        ('in', 1),
        ('wonderland', 1),
        ('wonderland', 1),
       ]

In [90]:
rdd_2 = sc.parallelize(data)

In [91]:
# in case of map output (k,v)
rdd_3 = rdd_2.reduceByKey(lambda a,b: a+b)

In [92]:
rdd_3.collect()

[('wonderland', 3), ('in', 2), ('adventures', 2), ('project', 1), ('alice', 2)]